In [1]:
%matplotlib tk
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os, glob, cv2
import numpy as np
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
train_path = '../dataset/images-half/'
classes = ["100", "106", "109", "118", "217"]

In [3]:
def load_data_cnn(train_path):
    images = []
    labels = []
    classes = [lists for lists in os.walk(train_path)][0][1]    
    for cls in classes:
        file_path = os.path.join(train_path, cls, "*g")
        for file in glob.glob(file_path):
            image = cv2.imread(file)
            image = cv2.resize(image, (432, 288), cv2.INTER_LINEAR)
            image = image[288-250:288-35, 55:390]
            image = cv2.resize(image, (int(215/5), int(215/5)), cv2.INTER_LINEAR)
            label = np.zeros(len(classes))
            label[classes.index(cls)] = 1.0
            images.append(image)
            labels.append(label) 
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

dataset = load_data_cnn(train_path)

In [4]:
X_train = []
X_train.append(dataset[0][:80])
X_train.append(dataset[0][100:180])
X_train.append(dataset[0][200:280])
X_train.append(dataset[0][300:380])
X_train.append(dataset[0][400:480])
X_train = [j for i in X_train for j in i]
X_train = np.array(X_train)

y_train = []
y_train.append(dataset[1][:80])
y_train.append(dataset[1][100:180])
y_train.append(dataset[1][200:280])
y_train.append(dataset[1][300:380])
y_train.append(dataset[1][400:480])
y_train = [j for i in y_train for j in i]
y_train = np.array(y_train)

X_test = []
X_test.append(dataset[0][80:100])
X_test.append(dataset[0][180:200])
X_test.append(dataset[0][280:300])
X_test.append(dataset[0][380:400])
X_test.append(dataset[0][480:500])
X_test = [j for i in X_test for j in i]
X_test = np.array(X_test)

y_test = []
y_test.append(dataset[1][80:100])
y_test.append(dataset[1][180:200])
y_test.append(dataset[1][280:300])
y_test.append(dataset[1][380:400])
y_test.append(dataset[1][480:500])
y_test = [j for i in y_test for j in i]
y_test = np.array(y_test)

y_test_label = []
for i in range(100):
    if i >= 80:
        y_test_label.append(4)
    elif i >= 60:
        y_test_label.append(3)
    elif i >= 40:
        y_test_label.append(2)
    elif i >= 20:
        y_test_label.append(1)
    else:
        y_test_label.append(0)
        

0 - normal <br>
1 - paced <br>
2 - lbbb <br>
3 - pvc <br>
4 - rbbb

plt.imshow(X_train[320])

In [5]:
input_shape = (int(215/5), int(215/5), 3)

In [6]:
cnn_model = Sequential()

cnn_model.add(Conv2D(64, (3, 3), input_shape=input_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3)))
cnn_model.add(BatchNormalization())
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3)))
cnn_model.add(BatchNormalization())
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(5, activation='softmax'))

In [7]:
cnn_model.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
classifier = cnn_model.fit(X_train, y_train, validation_split=0.1, epochs=50, batch_size=9, shuffle=True)

Train on 360 samples, validate on 40 samples
Epoch 1/50
360/360 [==============================] - 2s 4ms/step - loss: 2.4028 - acc: 0.2222 - val_loss: 1.6228 - val_acc: 0.1500
Epoch 2/50
360/360 [==============================] - 0s 937us/step - loss: 1.6474 - acc: 0.3583 - val_loss: 1.3712 - val_acc: 0.4000
Epoch 3/50
360/360 [==============================] - 0s 921us/step - loss: 1.3632 - acc: 0.4972 - val_loss: 1.0955 - val_acc: 0.7000
Epoch 4/50
360/360 [==============================] - 0s 938us/step - loss: 1.2894 - acc: 0.5167 - val_loss: 0.7885 - val_acc: 0.9250
Epoch 5/50
360/360 [==============================] - 0s 961us/step - loss: 1.0131 - acc: 0.6194 - val_loss: 0.6763 - val_acc: 0.9500
Epoch 6/50
360/360 [==============================] - 0s 951us/step - loss: 1.0047 - acc: 0.6194 - val_loss: 0.5282 - val_acc: 0.9750
Epoch 7/50
360/360 [==============================] - 0s 974us/step - loss: 0.7622 - acc: 0.7222 - val_loss: 0.3859 - val_acc: 0.9750
Epoch 8/50
360/360 

In [9]:
loss, accuracy = cnn_model.evaluate(X_test, y_test)
print("Loss = "+str(loss))
print("Accuracy = "+str(accuracy))

100/100 [==============================] - 0s 281us/step
Loss = 0.07839754950255155
Accuracy = 0.97


In [10]:
acc_values = cnn_model.history.history['acc']
val_acc_values = cnn_model.history.history['val_acc']
epochs = range(1, len(acc_values) + 1)
plt.plot(epochs, acc_values, 'r', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [11]:
predictions = cnn_model.predict_classes(X_test, batch_size=5)

In [12]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [13]:
cn = confusion_matrix(y_test_label, predictions)

In [14]:
print ("Confusion Matrix")
cn

Confusion Matrix


array([[19,  0,  0,  1,  0],
       [ 1, 19,  0,  0,  0],
       [ 0,  0, 20,  0,  0],
       [ 0,  0,  0, 19,  1],
       [ 0,  0,  0,  0, 20]])

In [15]:
cnn_model.save("../models/cnn_5labels_7.h5")

In [2]:
from keras.models import load_model 

In [3]:
cnn_model = load_model("../models/cnn_5labels_7.h5")

In [4]:
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 41, 41, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 41, 41, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 41, 41, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 32)        18464     
_________________________________________________________________
batch_normalization_2 (Batch (None, 18, 18, 32)        128       
__________